In [2]:
# ==========================================================
# High-Fidelity Artwork Analyzer v9.4.1 (Fixed & Complete)
# ==========================================================
# Fully local — OpenCV, NumPy, Pillow only
# - Full resolution (no downscale)
# - Step-by-step timing diagnostics
# - Always-visible dark legend
# - Safe gamma normalization (no whiteout / blackout)
# ==========================================================

import os, cv2, numpy as np, time
from PIL import Image
from math import atan2, degrees

# ----------------------------------------------------------
# Utilities
# ----------------------------------------------------------
def load_rgb(path):
    print(f"\n🖼️ Loading image: {path}")
    img = Image.open(path).convert("RGB")
    arr = np.array(img)
    print(f"Image loaded — shape: {arr.shape}, dtype: {arr.dtype}")
    return arr

def ensure_dir(p): os.makedirs(p, exist_ok=True)
def to_uint8(img): return np.clip(img*255.0,0,255).astype(np.uint8)

# ----------------------------------------------------------
# Color Analysis
# ----------------------------------------------------------
def analyze_color_opencv(img, k=6):
    t0 = time.time()
    H,W=img.shape[:2]
    lab=cv2.cvtColor(img,cv2.COLOR_RGB2LAB)
    Z=lab.reshape(-1,3).astype(np.float32)
    crit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,50,0.3)
    _r,labels,centers=cv2.kmeans(Z,k,None,crit,10,cv2.KMEANS_PP_CENTERS)
    lbl2=labels.reshape(H,W)
    rgb_centers=cv2.cvtColor(centers.reshape(-1,1,3).astype(np.uint8),cv2.COLOR_Lab2RGB).reshape(-1,3)
    masks=[(lbl2==i).astype(np.uint8)*255 for i in range(k)]
    a,b=centers[:,1]-128,centers[:,2]-128
    hues=(np.degrees(np.arctan2(b,a))+360)%360
    warm=((hues<90)|(hues>330)).sum()/len(hues)
    cool=((hues>180)&(hues<300)).sum()/len(hues)
    harmony="mixed"
    if np.ptp(hues)<35: harmony="analogous"
    elif any(abs(hues[i]-hues[j])%360>150 for i in range(len(hues)) for j in range(i+1,len(hues))):
        harmony="complementary"
    print(f"🎨 Color clustering complete in {time.time()-t0:.2f}s — {k} clusters, harmony={harmony}")
    return dict(
        palette_rgb=[tuple(map(int,c)) for c in rgb_centers],
        masks=masks,
        color_variability=float(np.std(centers,0).mean()),
        warm_ratio=float(warm),
        cool_ratio=float(cool),
        harmony=harmony
    )

# ----------------------------------------------------------
# Texture, Saliency, Composition, Illumination
# ----------------------------------------------------------
def analyze_texture(img):
    t0=time.time()
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    lap=cv2.Laplacian(g,cv2.CV_64F)
    val=float(lap.var())
    print(f"🪶 Texture computed in {time.time()-t0:.2f}s — Laplacian variance={val:.2f}")
    return dict(laplacian_var=val)

def spectral_saliency(grayf):
    fft=np.fft.fft2(grayf)
    loga=np.log(np.abs(fft)+1e-8)
    spec=np.exp((loga-cv2.blur(loga,(3,3)))+1j*np.angle(fft))
    sal=np.abs(np.fft.ifft2(spec))**2
    sal=cv2.GaussianBlur(sal,(9,9),2)
    sal=(sal-sal.min())/(sal.max()-sal.min()+1e-6)
    return (sal*255).astype(np.uint8)

def subject_from_saliency(sal):
    H,W=sal.shape
    _,th=cv2.threshold(sal,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cnts,_=cv2.findContours(th,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if cnts:
        x,y,w,h=cv2.boundingRect(max(cnts,key=cv2.contourArea))
    else: x,y,w,h=W//4,H//4,W//2,H//2
    mask=np.zeros((H,W),np.uint8); cv2.rectangle(mask,(x,y),(x+w,y+h),255,-1)
    return dict(mask=mask,bbox=(x,y,w,h))

def analyze_saliency(img):
    t0=time.time()
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY).astype(np.float32)
    sal=spectral_saliency(g)
    sub=subject_from_saliency(sal)
    Y,X=np.indices(sal.shape); w=sal.astype(np.float32)+1
    cx,cy=int((X*w).sum()/w.sum()),int((Y*w).sum()/w.sum())
    print(f"🔥 Saliency + subject done in {time.time()-t0:.2f}s — focal=({cx},{cy})")
    return dict(saliency_map=sal,focal=(cx,cy),
                subject_mask=sub["mask"],subject_bbox=sub["bbox"])

def analyze_composition(img):
    t0=time.time()
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    H,W=g.shape
    edges=cv2.Canny(g,100,200)
    linesP=cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=min(H,W)//10,maxLineGap=15)
    lines=[tuple(map(int,l[0])) for l in linesP] if linesP is not None else []
    lw,rw=float(np.sum(edges[:,:W//2])),float(np.sum(edges[:,W//2:]))
    bal=(rw-lw)/(lw+rw+1e-6)
    print(f"📐 Composition lines={len(lines)} | balance={bal:.3f} | time={time.time()-t0:.2f}s")
    return dict(lines=lines,edge_balance=bal,
                vanishing_point=(W//2,H//2))

def illumination(img):
    t0=time.time()
    L=cv2.cvtColor(img,cv2.COLOR_RGB2LAB)[:,:,0].astype(np.float32)
    mu,sig=L.mean(),L.std()+1e-6
    skew=float((((L-mu)/sig)**3).mean())
    gx,gy=cv2.Sobel(L,cv2.CV_32F,1,0),cv2.Sobel(L,cv2.CV_32F,0,1)
    hi=(L>np.percentile(L,90)).astype(np.uint8)
    if hi.sum()>0:
        gx,gy=(gx*hi).sum(),(gy*hi).sum()
        ang=float((degrees(atan2(-gy,-gx))+360)%360)
    else: ang=None
    print(f"💡 Illumination in {time.time()-t0:.2f}s — skew={skew:.3f}, angle={ang}")
    return dict(lightness_skew=skew,light_direction_deg=ang)

# ----------------------------------------------------------
# Visualization
# ----------------------------------------------------------
def make_composite(img,metrics):
    base=img.astype(np.float32)/255.0
    H,W=base.shape[:2]

    mean_brightness=base.mean()
    blend_scale=0.7 if mean_brightness>0.6 else 1.1 if mean_brightness<0.4 else 0.9

    color_layer=np.zeros_like(base)
    for m,c in zip(metrics["color"]["masks"],metrics["color"]["palette_rgb"]):
        if m.sum()==0: continue
        mask=cv2.GaussianBlur((m>0).astype(np.float32),(0,0),6)
        color_layer+=np.dstack([mask]*3)*(np.array(c)/255.0)
    color_layer*=0.25

    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    lap=np.abs(cv2.Laplacian(g,cv2.CV_32F))
    lap=cv2.normalize(lap,None,0,1,cv2.NORM_MINMAX)
    tex_mask=(lap>np.percentile(lap,98)).astype(np.float32)
    tex_layer=np.dstack([tex_mask]*3)*[1.0,0.4,0.0]*0.25

    sal=metrics["saliency"]["saliency_map"].astype(np.float32)
    sal=cv2.GaussianBlur(sal,(0,0),4)
    sal=cv2.normalize(sal,None,0,1,cv2.NORM_MINMAX)
    sal_layer=np.dstack([sal*0.6,sal*0.2,sal*0.2])*0.3

    line_layer=np.zeros_like(base)
    for (x1,y1,x2,y2) in metrics["composition"]["lines"][:100]:
        ang=(np.rad2deg(np.arctan2(y2-y1,x2-x1))+180)%180
        col=(0,1,0) if 60<ang<120 else (0,0,1) if ang<30 or ang>150 else (1,1,0)
        cv2.line(line_layer,(x1,y1),(x2,y2),(col[2],col[1],col[0]),1,cv2.LINE_AA)
    line_layer=cv2.GaussianBlur(line_layer,(0,0),2)*0.4

    comp=base*blend_scale + color_layer + tex_layer + sal_layer + line_layer
    comp=np.clip(comp,0,1)

    mask=metrics["saliency"]["subject_mask"]
    cnts,_=cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    comp_bgr=(comp[:,:,::-1]*255).astype(np.uint8).copy()
    if cnts: cv2.drawContours(comp_bgr,cnts,-1,(255,0,255),2)
    fx,fy=metrics["saliency"]["focal"]
    for r in range(5,25,5): cv2.circle(comp_bgr,(int(fx),int(fy)),r,(0,0,255),1)

    vp=metrics["composition"].get("vanishing_point")
    if vp: cv2.circle(comp_bgr,tuple(map(int,vp)),6,(255,255,0),-1)
    ang=metrics["illumination"].get("light_direction_deg")
    if ang is not None:
        cx,cy=W-60,60; r=min(H,W)//3
        ex,ey=int(cx - r*np.cos(np.radians(ang))),int(cy + r*np.sin(np.radians(ang)))
        cv2.arrowedLine(comp_bgr,(cx,cy),(ex,ey),(0,150,255),3,tipLength=0.1)

    # --- Dark legend background ---
    legend_h=90
    legend=np.zeros((legend_h,W,3),np.float32)
    font=cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(legend,"Legend:",(20,55),font,0.8,(255,255,255),2)
    sw=[((110,30),(150,60),(0,0,1),"Focal"),
        ((220,30),(260,60),(1,0,1),"Subject"),
        ((330,30),(370,60),(0,1,0),"Horiz"),
        ((440,30),(480,60),(1,1,0),"Diag"),
        ((550,30),(590,60),(1,0.5,0),"Texture"),
        ((660,30),(700,60),(0.8,0.8,0.8),"Color")]
    if vp: sw.append(((800,30),(840,60),(0,1,1),"Vanishing"))
    if ang is not None: sw.append(((940,30),(980,60),(1,0.6,0.2),"Light→"))
    for (x1,y1),(x2,y2),col,label in sw:
        c=tuple(int(v*255) for v in col)
        cv2.rectangle(legend,(x1,y1),(x2,y2),c,-1)
        cv2.putText(legend,label,(x2+10,y2-5),font,0.6,(255,255,255),1)

    final = np.vstack([(comp_bgr[:,:,::-1]/255.0), legend])

    # --- Safe visualization normalization ---
    final = np.clip(final, 0, 1)
    if final.mean() < 0.2:
        final = np.power(final, 0.6)  # brighten darks
    elif final.mean() > 0.8:
        final = np.power(final, 1.2)  # compress brights

    return final

# ----------------------------------------------------------
# Pipeline
# ----------------------------------------------------------
def analyze_artwork_high_fidelity(path):
    start=time.time()
    img=load_rgb(path)
    color=analyze_color_opencv(img)
    tex=analyze_texture(img)
    sal=analyze_saliency(img)
    comp=analyze_composition(img)
    illum=illumination(img)
    metrics=dict(color=color,texture=tex,saliency=sal,composition=comp,illumination=illum)
    compo=make_composite(img,metrics)
    out_dir=os.path.splitext(path)[0]+"_analysis"
    ensure_dir(out_dir)
    out_png=os.path.join(out_dir,"overlay_final_composite.png")
    cv2.imwrite(out_png,to_uint8(compo))
    print(f"\n✅ Saved composite: {out_png}")
    print(f"⏱️ Total analysis time: {time.time()-start:.2f}s")

# ----------------------------------------------------------
# Run Example
# ----------------------------------------------------------
if __name__=="__main__":
    IMAGE_PATH="/Users/alievanayasso/Documents/SlowMA/Rembrandt.jpg"
    analyze_artwork_high_fidelity(IMAGE_PATH)



🖼️ Loading image: /Users/alievanayasso/Documents/SlowMA/Rembrandt.jpg
Image loaded — shape: (1477, 2200, 3), dtype: uint8
🎨 Color clustering complete in 2.84s — 6 clusters, harmony=analogous
🪶 Texture computed in 0.01s — Laplacian variance=106.17
🔥 Saliency + subject done in 0.23s — focal=(1074,786)
📐 Composition lines=4 | balance=-0.110 | time=0.01s
💡 Illumination in 0.03s — skew=1.238, angle=65.52912020454437

✅ Saved composite: /Users/alievanayasso/Documents/SlowMA/Rembrandt_analysis/overlay_final_composite.png
⏱️ Total analysis time: 3.77s
